### 1. Програма для отримання радомної анкети

In [2]:
import requests
from bs4 import BeautifulSoup



site = 'https://www.i.ua/'

r = requests.get(site)
r.status_code

200

In [4]:
soup = BeautifulSoup(r.text, 'lxml')
type(soup)

bs4.BeautifulSoup

In [8]:
quest = soup.find('div', class_="block_gamma_gradient photo")

Отримаємо імя, вік, фото, посилання на профіль

In [12]:
name = quest.p.text

In [25]:
age = int(quest.dd.text.split('\n')[2].split(',')[0].split()[0])

In [33]:
link = quest.dt.a.get('href')

In [41]:
photo = quest.dt.a.img.get('src')

In [50]:
r = requests.get('http:'+photo)
with open(name+'.jpeg', 'wb') as f:
    f.write(r.content)

### 2. Випадкові анкети із сайту з детальним описом

In [95]:
import requests
from bs4 import BeautifulSoup

# HEADERS = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
#             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#             'Accept-Language': 'ua-UA,en;q=0.5',
#             'Accept-Encoding': 'gzip, deflate',
#             'DNT': '1',
#             'Connection': 'keep-alive',
#             'Upgrade-Insecure-Requests': '1'
#         }

site = 'https://love.i.ua/'

r = requests.get(site, headers=HEADERS)
soup = BeautifulSoup(r.text, 'lxml')

In [97]:
for teg in soup.find_all('h2', class_="underlined"):
    if teg.text == 'Случайные анкеты':
        break
teg

<h2 class="underlined">Случайные анкеты</h2>

In [98]:
DB = {}
for q in teg.parent.find_all('ul', class_="list_cols_5 clear")[1].find_all('li'):
    name = q.p.text
    link = q.a.get('href').strip('/')
    print(site+link)
    describe = get_describe(site+link)
    DB[link] = {'name':name, 'describe':describe}

https://love.i.ua/user/1052269
https://love.i.ua/user/889771
https://love.i.ua/user/8476050
https://love.i.ua/user/1086337
https://love.i.ua/user/4468387


In [99]:
def get_describe(link):
    '''Отримуємо опис анкети користувача'''
    
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        result = soup.find('div', id="name_block").text
    except AttributeError:
        result = 'Відсутній опис'
    
    return result

In [100]:
DB

{'user/1052269': {'name': 'Віталій',
  'describe': 'Постійно в роботі, шукаю другу полинку.'},
 'user/889771': {'name': 'S_T_E',
  'describe': 'Увлечения - чтение, танцы, музыка'},
 'user/8476050': {'name': 'Алексей',
  'describe': 'Полтавчанин,44х185х80,раз ведён.Живу один, без проблем,авто,бизнес,добры й оптимист,с чувством юмора,не курю могу выпить на празднике'},
 'user/1086337': {'name': 'Елена',
  'describe': 'Я просто человек со сложным характером. Я не грубиянка, но могу сказать лишнего… Я не верю в чудеса,но жду их..каждый день и каждую минуту..\r\n Мне пофиг, что вы про меня думаете — моя бабушка говорит, что я СОЛНЫШКО! :)\n\r\nЖенщины созданы для того, чтобы их любили, а не для того, чтобы их понимали (Оскар Уайльд)\n\r\nТы когда-нибудь чувствовал, что тебе не хватает того, кого ты никогда не встречал?'},
 'user/4468387': {'name': 'ПАВЛИНА', 'describe': 'Живу! Учусь! Работаю!'}}

In [101]:
id(DB)

2823551282616

### 3.  Парсинг сайту з фільмами

In [131]:
import requests
from bs4 import BeautifulSoup

# HEADERS = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
#             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#             'Accept-Language': 'ua-UA,en;q=0.5',
#             'Accept-Encoding': 'gzip, deflate',
#             'DNT': '1',
#             'Connection': 'keep-alive',
#             'Upgrade-Insecure-Requests': '1'
#         }

site = 'https://www.ivi.tv/'

def get_genre(site):
    r = requests.get(site, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')

    dict_with_genre = {}
    for el in soup.find_all('li', class_="top-menu__sublist-item"):
        dict_with_genre[el.a.text] = el.a.get('href')
    
    print(list(dict_with_genre))
    
    return dict_with_genre

def get_film(site, genre):
    r = requests.get(site+genre.strip('/'), headers=HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')

    dict_with_film = {}
    for film in soup.find_all('li', class_="gallery__item gallery__item_virtual")[0:5]:
        name = film.find('div', class_="nbl-slimPosterBlock__title").text
        link = film.a.get('href')
        dict_with_film[name] = link

    return dict_with_film

def detail_info(site, film):
    r = requests.get(site+film.strip('/'), headers=HEADERS)
    soup = BeautifulSoup(r.text, 'lxml')
    
    info = soup.find('div', class_="clause__text-inner").p.text
    
    print(info)

if __name__ == '__main__':
    
    all_genre = get_genre(site)
    g = input('Введіть назву жанру')
    
    dict_with_film = get_film(site, all_genre[g])
    
    print(list(dict_with_film))
    
    film = input('Введіть назву фільму: ')
    
    detail_info(site, dict_with_film[film])
    
    
    

['Артхаус', 'Боевики', 'Военные', 'Детективы', 'Для всей семьи', 'Для детей', 'Документальные', 'Драмы', 'Исторические', 'Комедии', 'Криминал', 'Мелодрамы', 'Приключения', 'Триллеры', 'Ужасы', 'Фантастика', 'Фэнтези', '\n                        Новинки подписки\n                    ', 'Комедийные', 'Медицинские', 'Романтические', '\n                        Передачи National Geographic\n                    ', '\n                        Сериалы, телешоу и фильмы Первого канала\n                    ', 'Для взрослых', 'Полнометражные', 'Развивающие', 'Сериалы']
Введіть назву жанруПриключения
['Фиксики: Большой секрет', 'Три богатыря и Наследница престола', 'Оранжевая корова', 'Три богатыря на дальних берегах', 'Три богатыря и Шамаханская царица']
Введіть назву фільму: Фиксики: Большой секрет
Фиксики, маленькие герои одноименного мультсериала, наконец-то появились и на большом экране: о них сняли полнометражный мультфильм. 

Крохотные волшебные существа живут в обычных человеческих домах и 